In [ ]:
%%time
!wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O ml-1m.zip
!unzip -o ml-1m.zip
!rm ml-1m.zip

In [4]:
import copy
from pprint import pprint

import numpy as np
import pandas as pd

import time
from tqdm.auto import tqdm

from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.metrics.ranking import MAP, MRR
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.models import RandomModel, PopularModel
from rectools.model_selection import TimeRangeSplitter

In [5]:
%%time
dataset = pd.read_csv("data_original/interactions.csv")
dataset.head()

CPU times: user 1.22 s, sys: 166 ms, total: 1.39 s
Wall time: 1.39 s


,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [10]:
dataset = dataset.rename(columns={'last_watch_dt': 'datetime', 'watched_pct': 'weight'})
interactions = Interactions(dataset)
interactions.df.head()

,user_id,item_id,datetime,total_dur,weight
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [30]:
n_splits = 3

cv = TimeRangeSplitter(
    test_size="14D",
    n_splits=n_splits,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

models = {
    "random": RandomModel(random_state=32),
    "popular": PopularModel()
}

metrics = {
    "prec@1": Precision(k=1),
    "prec@5": Precision(k=5),
    "prec@10": Precision(k=10),
    "recall@1": Recall(k=1),
    "recall@5": Recall(k=5),
    "recall@10": Recall(k=10),

    "novelty@1": MeanInvUserFreq(k=1),
    "novelty@5": MeanInvUserFreq(k=5),
    "novelty@10": MeanInvUserFreq(k=10),
    "serendipity@1": Serendipity(k=1),
    "serendipity@5": Serendipity(k=5),
    "serendipity@10": Serendipity(k=10),

    "MAP@1": MAP(k=1),
    "MAP@5": MAP(k=5),
    "MAP@10": MAP(k=10),
    "MRR@1": MRR(k=1),
    "MRR@5": MRR(k=5),
    "MRR@10": MRR(k=10),

}

K_RECOS = 10

In [31]:
def metrics_validation(splitter, interactions, models: dict, metrics: dict, k: int):
    
    results = []

    fold_iterator = splitter.split(interactions, collect_fold_stats=True)

    for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=n_splits):
        print(f"\n==================== Fold {fold_info['i_split']}")
        # pprint(fold_info)

        df_train = interactions.df.iloc[train_ids]
        dataset = Dataset.construct(df_train)

        df_test = interactions.df.iloc[test_ids][Columns.UserItem]
        test_users = np.unique(df_test[Columns.User])

        catalog = df_train[Columns.Item].unique()

        for model_name, model in models.items():
            
            start_fit = time.time()
            model.fit(dataset)
            print('train time = ', time.time() - start_fit)

            models[model_name] = copy.deepcopy(model)

            recos = model.recommend(
                users=test_users,
                dataset=dataset,
                k=k,
                filter_viewed=True,
            )
            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )
            res = {"fold": fold_info["i_split"], "model": model_name}
            res.update(metric_values)
            results.append(res)
    return results

In [32]:
metrics_list = metrics_validation(cv, interactions=interactions, models=models, metrics=metrics, k=K_RECOS)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0
train time =  4.363059997558594e-05
train time =  0.9574792385101318


 33%|███▎      | 1/3 [00:18<00:36, 18.34s/it]


==================== Fold 1
train time =  3.504753112792969e-05
train time =  1.0932250022888184


 67%|██████▋   | 2/3 [00:39<00:19, 19.98s/it]


==================== Fold 2
train time =  3.337860107421875e-05
train time =  1.332792043685913


100%|██████████| 3/3 [01:04<00:00, 21.47s/it]


In [37]:
metrics_df = pd.DataFrame(metrics_list)
metrics_df

In [40]:
metrics_df_mean = metrics_df.groupby('model').mean()
metrics_df_mean

,fold,prec@1,recall@1,prec@5,recall@5,prec@10,recall@10,MRR@1,MRR@5,MRR@10,MAP@1,MAP@5,MAP@10,novelty@1,novelty@5,novelty@10,serendipity@1,serendipity@5,serendipity@10
model,,,,,,,,,,,,,,,,,,,
popular,1.0,0.097211,0.047186,0.070339,0.160496,0.045964,0.203517,0.097211,0.168272,0.176720,0.047186,0.091013,0.098549,2.422735,3.109863,3.722852,0.000003,0.000004,0.000003
random,1.0,0.000203,0.000055,0.000225,0.000293,0.000229,0.000634,0.000203,0.000499,0.000647,0.000055,0.000135,0.000179,15.567096,15.555586,15.557718,0.000008,0.000008,0.000008


In [41]:
metrics_df_mean.to_csv('Отчет_об_обучении.csv')